# Data Centre Modelling
## Ryan Jenkinson

### Imports

In [2]:
from dataclasses import dataclass

import numpy as np
import pandas as pd
import plotly.express as px
import pypsa
import xarray as xr

In [3]:
np.random.seed(42)  # for reproducibility

### Constants

In [4]:
@dataclass
class Costs:
    capital_cost: float
    marginal_cost: float

### Cost Assumptions

In [5]:
## TODO: Sensitivity analysis for costs
from typing import Literal

cost_scenario: Literal["low", "medium", "high"] = "medium"

## Wind costs
# https://www.gov.uk/government/publications/electricity-generation-costs-2023
# Using 2025 costs
KW_TO_MW = 1000
pre_development_costs_gbp_per_mw = {
    "low": 60 * KW_TO_MW,
    "medium": 130 * KW_TO_MW,
    "high": 200 * KW_TO_MW,
}
construction_costs_gbp_per_mw = {
    "low": 1300 * KW_TO_MW,
    "medium": 1500 * KW_TO_MW,
    "high": 2000 * KW_TO_MW,
}
wind_infrastructure_costs_gbp = {
    "low": 56_000_000,
    "medium": 64_300_000,
    "high": 74_100_000,
}  # TODO: How does this get added into the model? Do we annuitise it? But the other costs are annuitised per MW... :thinking:
wind_insurance = 3000  # £ per MW per year (insurance cost)

wind_capex_per_mw = (
    pre_development_costs_gbp_per_mw[cost_scenario]
    + construction_costs_gbp_per_mw[cost_scenario]
    + wind_insurance
)  # £ per MW
wind_lifetime_years = 30
wind_discount_rate = 0.07
annuitised_wind_cost = (wind_capex_per_mw * wind_discount_rate) / (
    1 - (1 + wind_discount_rate) ** -wind_lifetime_years
)
wind_fixed_opex = 43300  # £ per MW per year (fixed O&M)
wind_variable_operation_and_management_mwh = 1  # £ per MWh_elec (non-fuel variable O&M)

connection_and_use_of_system_charges = 44800  # £ per MW per year (grid connection costs) [NOTE: Not used in model as a microgrid]
WIND_COSTS = Costs(
    capital_cost=annuitised_wind_cost + wind_fixed_opex,  # £ per MW per year
    marginal_cost=0.01,  # £ per MWh (Variable O&M, assumed small)
)

## Gas costs
# Assume OCGT (Open Cycle Gas Turbine) for simplicity.
# Fast Start-Up: Aeroderivative gas turbines, which are derived from jet engine technology, can ramp up to full power in a matter of minutes. This rapid response is crucial to bridge the gap between a utility power outage and the exhaustion of the facility's Uninterruptible Power Supply (UPS) battery backup.
# High Reliability
# Smaller Footprint: Compared to other power generation technologies with similar output, OCGTs have a relatively compact footprint, which is an important consideration for data centers where space can be at a premium.
# Lower Emissions than Diesel
# CCGTs are an alternative for highly efficient for continuous, baseload power generation but they have a much longer start-up time and a significantly larger and more complex footprint so we use OCGT costs here
# https://www.gov.uk/government/publications/electricity-generation-costs-2023

# To decide the costings we use the following methodology:
# Operating Hours: 500 hr vs. 2000 hr --> 500hr is more akin to "backup" generator. 2000hr is equivalent to a standard 8 hour workday every weekday. So we use the 500hr variant
# OCGT 100MW 500 hr -> Logical starting point. Most data centres use 10 x 10MW backups so modelling as a 100MW is a good proxy
# OCGT 299MW 500 hr --> common to model even larger data centres as after 300MW there is much more complicated regulations
# We use the OCGT 100MW

gas_reference_model: Literal["100MW", "299MW"] = (
    "100MW"  # Reference model for gas turbine costs
)

gas_fuel_efficiency = {
    "100MW": 0.34,  # Efficiency of the 100MW OCGT (MWh_elec / MWh_fuel)
    "299MW": 0.35,  # Efficiency of the 299MW OCGT (MWh_elec / MWh_fuel)
}
gas_pre_development_costs_gbp_per_mw = {
    "100MW": {"low": 80 * KW_TO_MW, "medium": 90 * KW_TO_MW, "high": 110 * KW_TO_MW},
    "299MW": {"low": 30 * KW_TO_MW, "medium": 40 * KW_TO_MW, "high": 40 * KW_TO_MW},
}
gas_construction_costs_gbp_per_mw = {
    "100MW": {"low": 600 * KW_TO_MW, "medium": 700 * KW_TO_MW, "high": 900 * KW_TO_MW},
    "299MW": {"low": 300 * KW_TO_MW, "medium": 500 * KW_TO_MW, "high": 800 * KW_TO_MW},
}
gas_infrastructure_costs_gbp = {
    "100MW": {"low": 7_200_000, "medium": 14_400_000, "high": 28_700_000},
    "299MW": {"low": 7_800_000, "medium": 15_500_000, "high": 31_000_000},
}  # TODO: How to add this into the model? Do we annuitise it? But the other costs are annuitised per MW... :thinking:
gas_insurance = {
    "100MW": 2900,  # £ per MW per year (insurance cost for 100MW OCGT)
    "299MW": 1800,  # £ per MW per year (insurance cost for 299MW OCGT)
}
gas_connection_use_of_system_charges = 2700  # NOTE: Not used in model as a microgrid

gas_capex_per_mw = (
    gas_pre_development_costs_gbp_per_mw[gas_reference_model][cost_scenario]
    + gas_construction_costs_gbp_per_mw[gas_reference_model][cost_scenario]
    + gas_insurance[gas_reference_model]
)
gas_lifetime_years = 25
gas_discount_rate = 0.07
annuitised_gas_cost = (gas_capex_per_mw * gas_discount_rate) / (
    1 - (1 + gas_discount_rate) ** -gas_lifetime_years
)
gas_fixed_opex = {
    "100MW": 11300,  # £ per MW per year (fixed O&M for 100MW OCGT)
    "299MW": 7300,  # £ per MW per year (fixed O&M for 299MW OCGT)
}[gas_reference_model]  # Fixed O&M cost based on the reference model
gas_fuel_price_mwh = 60  # £ per MWh_fuel (assumed fuel price) # TODO: Validate
gas_variable_operation_and_management_mwh = {
    "100MW": 2,  # £ per MWh_elec (non-fuel variable O&M for 100MW OCGT)
    "299MW": 1,  # £ per MWh_elec (non-fuel variable O&M for 299MW OCGT)
}[gas_reference_model]
gas_marginal_cost_fuel = (
    gas_fuel_price_mwh / gas_fuel_efficiency[gas_reference_model]
)  # £ per MWh_elec
GAS_COSTS = Costs(
    capital_cost=annuitised_gas_cost + gas_fixed_opex,  # £ per MW per year
    marginal_cost=gas_marginal_cost_fuel
    + gas_variable_operation_and_management_mwh,  # £ per MWh_elec
)

## BESS (Battery Energy Storage System) costs
# https://www.utilitydive.com/news/tariffs-to-spike-power-generation-costs-reports/750133/
# https://www.numberanalytics.com/blog/energy-storage-economics
bess_energy_capex_per_mwh = {
    "low": 140000,  # £ per MWh
    "medium": 170000,  # £ per MWh
    "high": 200000,  # £ per MWh
}
bess_power_capex_per_mw = {
    "low": 450000,  # £ per MW
    "medium": 540000,  # £ per MW
    "high": 630000,  # £ per MW
}
bess_lifetime_years = 15
bess_discount_rate = 0.07

annuitised_bess_energy_cost = (
    bess_energy_capex_per_mwh[cost_scenario] * bess_discount_rate
) / (1 - (1 + bess_discount_rate) ** -bess_lifetime_years)
bess_capital_cost_energy = (
    annuitised_bess_energy_cost  # Cost for energy capacity (Store)
)

annuitised_bess_power_cost = (
    bess_power_capex_per_mw[cost_scenario] * bess_discount_rate
) / (1 - (1 + bess_discount_rate) ** -bess_lifetime_years)
bess_fixed_opex_per_mw_power = 20000  # Fixed OPEX for power conversion part
bess_capital_cost_power = (
    annuitised_bess_power_cost + bess_fixed_opex_per_mw_power
)  # Cost for power capacity (Links)

bess_round_trip_efficiency = 0.87
standing_loss_per_hour = 0.001
bess_marginal_cost_links = 0.001  # Marginal cost for charging/discharging

BESS_COSTS = Costs(
    capital_cost=bess_capital_cost_energy,  # £ per MWh per year
    marginal_cost=0,
)
BESS_LINK_COSTS = Costs(
    capital_cost=bess_capital_cost_power,  # £ per MW per year
    marginal_cost=bess_marginal_cost_links,  # £ per MWh_elec
)
TOTAL_ANNUITISED_INFRASTRUCTURE_COSTS = (
    (wind_infrastructure_costs_gbp[cost_scenario] * wind_discount_rate) / (1 - (1 + wind_discount_rate) ** -wind_lifetime_years)
    + ((gas_infrastructure_costs_gbp[gas_reference_model][cost_scenario] * gas_discount_rate) / (1 - (1 + gas_discount_rate) ** -gas_lifetime_years))
)

In [6]:
# TODO: Put everything relative to this or do a plot that plots this at difference sizes of data centre
DATA_CENTRE_LOAD_MW = 115  # MW, constant demand including PUE

### Wind Availability

In [7]:
wind_data = xr.load_dataset(
    "/Users/katherine.shaw/Desktop/pypsa-eur copy/resources/profile_39_offwind-dc.nc",
)

In [8]:
wind_profile_df = wind_data["profile"].to_dataframe().reset_index()

wind_availability_list = wind_profile_df[wind_profile_df["bus"] == "GB2 0"]["profile"].to_numpy()

In [9]:
wind_availability_list

array([0.85647309, 0.85884475, 0.85858394, ..., 0.41351394, 0.41253353,
       0.48444452])

### Model

In [10]:
MODEL_TIME_FREQUENCY = pd.Timedelta("1h")  # Time frequency for the model
def gen_model(data_centre_load_mw: float):
    # Define simulation period: a full year, hourly
    snapshots = pd.date_range(
        start="2019-01-01 00:00", end="2019-12-31 23:00", freq=MODEL_TIME_FREQUENCY
    )
    n = pypsa.Network(snapshots=snapshots)

    # --- DEFINE CARRIERS ---
    n.add("Carrier", "AC")  # For AC buses
    n.add("Carrier", "gas", co2_emissions=0.185)  # tCO2/MWh_th, typical for natural gas
    n.add("Carrier", "offshore_wind")
    n.add("Carrier", "battery")  # Carrier for battery components

    # --- DEFINE BUSES ---
    n.add("Bus", "electricity_bus", carrier="AC")
    n.add("Bus", "bess_connection_bus", carrier="AC")

    # --- DEFINE LOAD ---
    # Load (Data Centre): The data centre is modelled as a load with a constant power demand.
    constant_demand_profile = pd.Series(data_centre_load_mw, index=n.snapshots)
    n.add(
        "Load", "data_centre_load", bus="electricity_bus", p_set=constant_demand_profile
    )

    # --- DEFINE GENERATORS ---
    # Generator (Offshore Wind Farm): The offshore wind farm's capacity is extendable.

    wind_availability_pu = pd.Series(wind_availability_list, index=n.snapshots)
    wind_capacity_factor = 0.61  # Target capacity factor for the wind farm
    wind_availability_pu = wind_capacity_factor * (
        wind_availability_pu / wind_availability_pu.mean()
    )
    wind_availability_pu = wind_availability_pu.clip(0, 1)

    n.add(
        "Generator",
        "offshore_wind_farm",
        bus="electricity_bus",
        carrier="offshore_wind",
        p_nom_extendable=True,  # Allow capacity to be extended. To cover what we need to for the data centre load
        capital_cost=WIND_COSTS.capital_cost,
        marginal_cost=WIND_COSTS.marginal_cost,
        p_max_pu=wind_availability_pu,
    )

    # Generator (Gas Turbine Backup): The gas turbine provides backup, and its capacity is also extendable.
    n.add(
        "Generator",
        "gas_turbine",
        bus="electricity_bus",
        carrier="gas",
        p_nom_extendable=True,  # Allow capacity to be extended to cover the data centre load
        committable=False,  # Cannot be both committable and extendable for LOPF capacity optimization. TODO: Check if this is correct
        capital_cost=GAS_COSTS.capital_cost,
        marginal_cost=GAS_COSTS.marginal_cost,
    )

    # --- DEFINE STORAGE (BESS) ---
    # Store component representing the energy capacity of the BESS, and each of the links for charging and discharging.
    n.add(
        "Store",
        "bess_storage",
        bus="bess_connection_bus",
        carrier="battery",
        e_nom_extendable=True,
        e_cyclic=True,
        standing_loss=standing_loss_per_hour,
        capital_cost=BESS_COSTS.capital_cost,
    )
    n.add(
        "Link",
        "bess_charger",
        bus0="electricity_bus",
        bus1="bess_connection_bus",
        carrier="battery",
        p_nom_extendable=True,
        capital_cost=BESS_LINK_COSTS.capital_cost,
        efficiency=bess_round_trip_efficiency**0.5,
        marginal_cost=BESS_LINK_COSTS.marginal_cost,
    )
    n.add(
        "Link",
        "bess_discharger",
        bus0="bess_connection_bus",
        bus1="electricity_bus",
        carrier="battery",
        p_nom_extendable=True,
        capital_cost=0,  # Assuming power CAPEX is fully on charger link, or split if preferred
        efficiency=bess_round_trip_efficiency**0.5,
        marginal_cost=BESS_LINK_COSTS.marginal_cost,
    )

    # --- CONSISTENCY CHECK & SOLVE ---

    # Perform consistency check
    try:
        n.consistency_check()
        print("Network consistency check passed before solving.")
    except Exception as e:
        print(f"Consistency check error before solving: {e}")

    return n

In [11]:
# Ensure you have a solver installed (e.g., cbc, glpk, gurobi)
solver_name = "cbc"  # or 'glpk', 'gurobi', etc.


def solve_model(n: pypsa.Network, solver_name: str, print_results: bool = True):
    """Solve the PyPSA network model using the specified solver.

    Parameters:
    - n: PyPSA Network object
    - solver_name: Name of the solver to use (e.g., 'cbc', 'glpk')
    - print_results: Whether to print the results after solving
    """
    try:
        # Solve the model using the optimize interface, common with linopy backend
        # The `solve_model` method handles the optimization.
        n.optimize(solver_name=solver_name)
        if not print_results:
            return n
        print("Model solved successfully.")

        # --- DISPLAY RESULTS ---
        print("\n--- Optimization Results ---")
        # Objective value is typically stored in n.model.objective after solving with n.optimize
        obj_value = n.model.objective.value
        print(f"Objective Value: {obj_value:.2f} £ (Total Cost)")

        print("\nOptimal Capacities:")
        # Check if components exist in results before trying to access them
        # Optimal capacities are in p_nom_opt (for Generators, Links) and e_nom_opt (for Stores)
        if (
            "offshore_wind_farm" in n.generators.index
            and "p_nom_opt" in n.generators.columns
        ):
            print(
                f"  Offshore Wind Farm (p_nom_opt): {n.generators.loc['offshore_wind_farm', 'p_nom_opt']:.2f} MW"
            )
        if "gas_turbine" in n.generators.index and "p_nom_opt" in n.generators.columns:
            print(
                f"  Gas Turbine (p_nom_opt): {n.generators.loc['gas_turbine', 'p_nom_opt']:.2f} MW"
            )
        if "bess_storage" in n.stores.index and "e_nom_opt" in n.stores.columns:
            print(
                f"  BESS Storage Energy Capacity (e_nom_opt): {n.stores.loc['bess_storage', 'e_nom_opt']:.2f} MWh"
            )
        if "bess_charger" in n.links.index and "p_nom_opt" in n.links.columns:
            print(
                f"  BESS Charger Power Capacity (p_nom_opt): {n.links.loc['bess_charger', 'p_nom_opt']:.2f} MW"
            )
        if "bess_discharger" in n.links.index and "p_nom_opt" in n.links.columns:
            print(
                f"  BESS Discharger Power Capacity (p_nom_opt): {n.links.loc['bess_discharger', 'p_nom_opt']:.2f} MW"
            )

    except Exception as e:
        print(f"An error occurred during model solution or results processing: {e}")
        print("Please ensure you have a compatible solver installed and in your PATH.")
        print("Common solvers: cbc, glpk. You might need to install them.")
        print(
            "Also ensure Linopy (PyPSA's modeling backend) is installed and functioning correctly."
        )
    return n

In [12]:
def get_optimal_capacities(n: pypsa.Network):
    """Get the optimal capacities of the network components after solving."""
    optimal_capacities = {
        "offshore_wind_farm": n.generators.loc["offshore_wind_farm", "p_nom_opt"],
        "gas_turbine": n.generators.loc["gas_turbine", "p_nom_opt"],
        "bess_storage": n.stores.loc["bess_storage", "e_nom_opt"],
        "bess_charger": n.links.loc["bess_charger", "p_nom_opt"],
        "bess_discharger": n.links.loc["bess_discharger", "p_nom_opt"],
    }
    return optimal_capacities


def get_asset_expenditure(n: pypsa.Network):
    stats = n.statistics()
    return {
        "gas": (
            stats.loc[("Generator", "gas"), "Capital Expenditure"]
            + stats.loc[("Generator", "gas"), "Operational Expenditure"]
        ),
        "offshore_wind": (
            stats.loc[("Generator", "offshore_wind"), "Capital Expenditure"]
            + stats.loc[("Generator", "offshore_wind"), "Operational Expenditure"]
        ),
        "battery": (
            stats.loc[("Link", "battery"), "Capital Expenditure"]
            + stats.loc[("Link", "battery"), "Operational Expenditure"]
            + stats.loc[("Store", "battery"), "Capital Expenditure"]
            + stats.loc[("Store", "battery"), "Operational Expenditure"]
        ),
    }

# Sensitivity analysis for different DC loads

In [13]:
networks = {}
DC_LOADS = [10, 40, 100, 115, 500]  # Different data centre load scenarios in MW

for i, dc_load_value in enumerate(DC_LOADS):
    n = gen_model(data_centre_load_mw=dc_load_value)
    n = solve_model(n, solver_name="gurobi", print_results=False)
    networks[dc_load_value] = n

Network consistency check passed before solving.


INFO:linopy.model: Solve problem using Gurobi solver
INFO:linopy.io:Writing objective.
Writing continuous variables.: 100%|██████████| 7/7 [00:00<00:00, 189.53it/s]
INFO:linopy.io: Writing time: 0.28s


Set parameter LicenseID to value 2648928


INFO:gurobipy:Set parameter LicenseID to value 2648928


Read LP format model from file /private/var/folders/_w/ry357l9n0vjg8jgfczvkpk3r0000gn/T/linopy-problem-_7a9tb2l.lp


...

INFO:gurobipy:


Solved in 15558 iterations and 0.59 seconds (1.36 work units)


INFO:gurobipy:Solved in 15558 iterations and 0.59 seconds (1.36 work units)


Optimal objective  3.240049084e+08


INFO:gurobipy:Optimal objective  3.240049084e+08
INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 52565 primals, 113885 duals
Objective: 3.24e+08
Solver model: available
Solver message: 2

INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Generator-ext-p-lower, Generator-ext-p-upper, Link-ext-p-lower, Link-ext-p-upper, Store-ext-e-lower, Store-ext-e-upper, Store-energy_balance were not assigned to the network.


In [14]:
capacities = {load: get_optimal_capacities(n) for load, n in networks.items()}
capacities_df = pd.DataFrame(capacities).T  # Transpose so loads are rows
capacities_df.index.name = "data_centre_load_MW"
capacities_df.reset_index(inplace=True)
capacities_df

,data_centre_load_MW,offshore_wind_farm,gas_turbine,bess_storage,bess_charger,bess_discharger
0,10,18.899108,9.140279,4.556892,0.214759,3.734599
1,40,75.596430,36.561115,18.227566,0.859036,14.938396
2,100,188.991076,91.402787,45.568915,2.147590,37.345990
3,115,217.339738,105.113205,52.404252,2.469728,42.947889
4,500,944.955380,457.013936,227.844576,10.737948,186.729951


In [15]:
fig = px.line(
    data_frame=capacities_df,
    x="data_centre_load_MW",
    y=capacities_df.columns,
    title="Generator Power Output per DC scenario",
    labels={"value": "Power (MW)", "variable": "Generator"},
    template="plotly_white",
)
fig.show()

In [16]:
costs = {load: n.objective for load, n in networks.items()}

In [17]:
costs_df = pd.DataFrame.from_dict(costs, orient="index")
costs_df.index.name = "data_centre_load_MW"
costs_df.reset_index(inplace=True)
costs_df.columns = [
    "data_centre_load_MW",
    "objective cost (GBP)",
]  # Rename columns for clarity
costs_df["cost_per_dc_mw"] = (
    costs_df["objective cost (GBP)"] / costs_df["data_centre_load_MW"]
)
costs_df

,data_centre_load_MW,objective cost (GBP),cost_per_dc_mw
0,10,6.480098e+06,648009.816887
1,40,2.592039e+07,648009.816887
2,100,6.480098e+07,648009.816887
3,115,7.452113e+07,648009.816887
4,500,3.240049e+08,648009.816887


In [18]:
asset_expenditures = {load: get_asset_expenditure(n) for load, n in networks.items()}
asset_expenditures_df = (
    pd.DataFrame(asset_expenditures).T
    .assign(infrastructure=TOTAL_ANNUITISED_INFRASTRUCTURE_COSTS)
)
asset_expenditures_df["total"] = asset_expenditures_df.sum(axis=1)
asset_expenditures_df = asset_expenditures_df.reset_index(names="data_centre_load_MW")
asset_expenditures_df

,data_centre_load_MW,gas,offshore_wind,battery,infrastructure,total
0,10,3.071861e+06,3.306154e+06,1.020837e+05,6.417377e+06,1.289748e+07
1,40,1.228744e+07,1.322461e+07,4.083350e+05,6.417377e+06,3.233777e+07
2,100,3.071861e+07,3.306154e+07,1.020837e+06,6.417377e+06,7.121836e+07
3,115,3.532640e+07,3.802077e+07,1.173963e+06,6.417377e+06,8.093851e+07
4,500,1.535930e+08,1.653077e+08,5.104187e+06,6.417377e+06,3.304223e+08


In [19]:
fig = px.line(
    data_frame=asset_expenditures_df,
    x="data_centre_load_MW",
    y=asset_expenditures_df.columns,
    title="Total Asset Expenditures per DC scenario",
    labels={"value": "Cost (GBP)", "variable": "Generator"},
    template="plotly_white",
)
fig.show()

# Single datacentre exploration

In [20]:
n = gen_model(data_centre_load_mw=DATA_CENTRE_LOAD_MW)
n = solve_model(n, solver_name="gurobi", print_results=True) #changed to gurobi from highs for Katie's runs because she has access to gurobi

INFO:linopy.model: Solve problem using Gurobi solver


Network consistency check passed before solving.


INFO:linopy.io:Writing objective.
Writing continuous variables.: 100%|██████████| 7/7 [00:00<00:00, 189.31it/s]
INFO:linopy.io: Writing time: 0.26s


Set parameter LicenseID to value 2648928


INFO:gurobipy:Set parameter LicenseID to value 2648928


...

Solved in 15842 iterations and 0.57 seconds (1.33 work units)


INFO:gurobipy:Solved in 15842 iterations and 0.57 seconds (1.33 work units)


Optimal objective  7.452112894e+07


INFO:gurobipy:Optimal objective  7.452112894e+07
INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 52565 primals, 113885 duals
Objective: 7.45e+07
Solver model: available
Solver message: 2

INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Generator-ext-p-lower, Generator-ext-p-upper, Link-ext-p-lower, Link-ext-p-upper, Store-ext-e-lower, Store-ext-e-upper, Store-energy_balance were not assigned to the network.


Model solved successfully.

--- Optimization Results ---
Objective Value: 74521128.94 £ (Total Cost)

Optimal Capacities:
  Offshore Wind Farm (p_nom_opt): 217.34 MW
  Gas Turbine (p_nom_opt): 105.11 MW
  BESS Storage Energy Capacity (e_nom_opt): 52.40 MWh
  BESS Charger Power Capacity (p_nom_opt): 2.47 MW
  BESS Discharger Power Capacity (p_nom_opt): 42.95 MW


In [21]:
generator_power = n.generators_t.p
data_centre_power = n.loads_t.p["data_centre_load"]
battery_power = n.links_t.p1["bess_charger"] - n.links_t.p1["bess_discharger"]

In [22]:
asset_power = generator_power.copy()
asset_power["bess"] = battery_power
asset_power["dc_load"] = data_centre_power
asset_power

Generator,offshore_wind_farm,gas_turbine,bess,dc_load
snapshot,,,,
2019-01-01 00:00:00,115.000000,0.000000,0.0,115.0
2019-01-01 01:00:00,115.000000,0.000000,0.0,115.0
2019-01-01 02:00:00,115.000000,0.000000,0.0,115.0
2019-01-01 03:00:00,115.000000,0.000000,0.0,115.0
2019-01-01 04:00:00,115.000000,0.000000,0.0,115.0
...,...,...,...,...
2019-12-31 19:00:00,67.502403,47.497597,0.0,115.0
2019-12-31 20:00:00,78.630132,36.369868,0.0,115.0
2019-12-31 21:00:00,93.401646,21.598354,0.0,115.0


In [23]:
fig = px.line(
    data_frame=asset_power,
    x=asset_power.index,
    y=asset_power.columns,
    title="Asset Power Output Over Time",
    labels={"value": "Power (MW)", "variable": "Asset"},
    template="plotly_white",
)
fig.show()

In [24]:
fig = px.line(
    data_frame=pd.DataFrame(battery_power),
    x=data_centre_power.index,
    y=0,
    title="BESS Power Output Over Time",
    labels={"value": "Power (MW)", "variable": "Generator"},
    template="plotly_white",
)
fig.show()

In [25]:
asset_energy_mwh_per_day = asset_power.groupby(pd.Grouper(freq="D")).apply(lambda x: x.abs().sum().mul(
    MODEL_TIME_FREQUENCY.total_seconds() / 3600
))  # Convert power to energy (MWh)
asset_energy_mwh_per_month = asset_power.groupby(pd.Grouper(freq="M")).apply(lambda x: x.abs().sum().mul(
    MODEL_TIME_FREQUENCY.total_seconds() / 3600
))  # Convert power to energy (MWh)
asset_energy_mwh_per_day.head()

/var/folders/_w/ry357l9n0vjg8jgfczvkpk3r0000gn/T/ipykernel_80734/1325227996.py:4: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.



Generator,offshore_wind_farm,gas_turbine,bess,dc_load
snapshot,,,,
2019-01-01,2780.178442,0.000000,18.821198,2760.0
2019-01-02,2600.729302,147.159520,82.916783,2760.0
2019-01-03,1489.258993,1270.741007,0.000000,2760.0
2019-01-04,2632.536827,132.402629,4.607218,2760.0
2019-01-05,2778.832833,0.000000,81.407244,2760.0


In [26]:
# For each time period in the day, create a stacked bar chart showing the amount of energy used from each of the generators to meet the data centre load
fig = px.bar(
    data_frame=asset_energy_mwh_per_day,
    x=asset_energy_mwh_per_day.index,
    y=list(set(asset_energy_mwh_per_day.columns.to_list()) - {"dc_load"}),
    title="Energy Used from Each Generator to Meet Data Centre Load on Each Day of the Year",
    labels={"value": "Power (MW)", "variable": "Generator"},
    template="plotly_white",
    color_discrete_map={
        "offshore_wind_farm": "lightgreen",
        "gas_turbine": "darkgrey",
        "bess": "orange"
    }
)
# Add a line for the data centre load
fig.add_scatter(
    x=asset_energy_mwh_per_day.index,
    y=asset_energy_mwh_per_day["dc_load"],
    mode="lines",
    name="Data Centre Load",
    line=dict(color="black", width=2),
)
fig.add_layout_image(
    dict(
        source="https://centrefornetzero.org/static/media/logo.7e2e2e2e.svg",  # SVG logo from their website
        xref="paper", yref="paper",
        x=1, y=1,  # Top-right corner
        sizex=0.2, sizey=0.2,
        xanchor="right", yanchor="top",
        layer="above"
    )
)
fig.show()

In [27]:
asset_energy_mwh_per_month

Generator,offshore_wind_farm,gas_turbine,bess,dc_load
snapshot,,,,
2019-01-31,70912.613502,14704.312078,702.053497,85560.0
2019-02-28,68912.388747,8417.504036,633.228379,77280.0
2019-03-31,79034.240562,6595.252081,883.551527,85560.0
2019-04-30,70018.724558,12839.173011,610.538858,82800.0
2019-05-31,57167.938296,28473.192716,701.496331,85560.0
2019-06-30,68142.445382,14712.358752,1079.287662,82800.0
2019-07-31,67239.513619,18395.410709,1027.609585,85560.0
2019-08-31,71589.521468,14046.130275,784.296001,85560.0
2019-09-30,71308.743060,11544.525850,847.736170,82800.0


In [28]:
# Add percentage columns for each generator
for col in ["offshore_wind_farm", "gas_turbine", "bess"]:
    asset_energy_mwh_per_month[f"{col}_pct"] = (
        asset_energy_mwh_per_month[col] / asset_energy_mwh_per_month["dc_load"] * 100
    ).round(1).astype(str) + "%"

# Melt the DataFrame for easier plotting with text labels
melted = asset_energy_mwh_per_month.reset_index().melt(
    id_vars=["snapshot"],
    value_vars=["offshore_wind_farm", "gas_turbine", "bess"],
    var_name="Generator",
    value_name="Energy (MWh)",
)
# Add percentage labels
melted["Percentage"] = [
    asset_energy_mwh_per_month.loc[row["snapshot"], f"{row['Generator']}_pct"]
    for _, row in melted.iterrows()
]

fig = px.bar(
    melted,
    x="snapshot",
    y="Energy (MWh)",
    color="Generator",
    title="Energy Used from Each Generator to Meet Data Centre Load on Each Month of the Year",
    labels={"snapshot": "Month"},
    template="plotly_white",
    color_discrete_map={
        "offshore_wind_farm": "lightgreen",
        "gas_turbine": "darkgrey",
        "bess": "orange"
    },
    text="Percentage"
)
# Add a line for the data centre load
fig.add_scatter(
    x=asset_energy_mwh_per_month.index,
    y=asset_energy_mwh_per_month["dc_load"],
    mode="lines",
    name="Data Centre Load",
    line=dict(color="black", width=2),
)

fig.show()

In [29]:
# Get optimised powers for each asset
wind_opt_power = n.generators.loc["offshore_wind_farm", "p_nom_opt"]
gas_opt_power = n.generators.loc["gas_turbine", "p_nom_opt"]
bess_input_power = n.links.loc["bess_charger", "p_nom_opt"]
bess_output_power = n.links.loc["bess_discharger", "p_nom_opt"]
bess_avg_power = (bess_input_power + bess_output_power) / 2
bess_energy_capacity = n.stores.loc["bess_storage", "e_nom_opt"]

# Calculate percentages of DC load
wind_pct = wind_opt_power / DATA_CENTRE_LOAD_MW * 100
gas_pct = gas_opt_power / DATA_CENTRE_LOAD_MW * 100
bess_pct = bess_avg_power / DATA_CENTRE_LOAD_MW * 100

percentages = [wind_pct, gas_pct, bess_pct]
labels = [
    "<b>[Offshore Wind]</b><br><i>Power:</i> {:.1f} MW<br><i>({:.1f}%)</i>".format(wind_opt_power, wind_pct),
    "<b>[Gas Turbine]</b><br><i>Power:</i> {:.1f} MW<br><i>({:.1f}%)</i>".format(gas_opt_power, gas_pct),
    "<b>[BESS]</b><br><i>Avg Power (Import/Export):</i> {:.1f} MW<br><i>Energy:</i> {:.1f} MWh<br><i>({:.1f}%)</i>".format(bess_avg_power, bess_energy_capacity, bess_pct)
]

fig = px.bar(
    x=percentages,
    y=["Data Centre Load"] * 3,
    orientation="h",
    text=labels,
    color=["offshore_wind_farm", "gas_turbine", "bess"],
    color_discrete_map={
        "offshore_wind_farm": "lightgreen",
        "gas_turbine": "darkgrey",
        "bess": "orange"
    },
    title="Optimised Microgrid Generation Powers as Percentage of Data Centre Load<br><i>(Assuming 115 MW DC Load)</i>",
    labels={"x": "Percentage of DC Load (%)", "y": ""},
)

# Add black outline to each bar
for trace in fig.data:
    trace.marker.line.width = 2
    trace.marker.line.color = "black"

fig.update_traces(texttemplate="%{text}", textposition="inside")
fig.update_layout(
    barmode="stack",
    xaxis_title="Percentage of DC Load (%)",
    yaxis_title="",
    template="plotly_white",
    showlegend=False,
)
fig.show()

In [30]:
total_gas_mwh_usage = n.generators_t.p["gas_turbine"].sum()
total_data_centre_mwh_usage = n.loads_t.p["data_centre_load"].sum()
print(
    f"Ratio of data centre load covered by gas turbine: {total_gas_mwh_usage / total_data_centre_mwh_usage:.2%}"
)

Ratio of data centre load covered by gas turbine: 15.01%


In [31]:
n.model.objective.value

74521128.94201323

## From Gemini (Nuclear PPA prices)
PPA Market Dynamics: The general PPA market in the UK has experienced volatility. While specific nuclear PPA prices for data centres are not public, the strike price for Hinkley Point C's Contract for Difference (CfD) is £92.50 per MWh (in 2012 prices), which serves as a long-term, inflation-indexed reference for new large-scale nuclear, albeit a subsidised rate. Market analysts have noted that premiums in the UK PPA market can add approximately £10/MWh to baseline PPA costs for corporate buyers seeking specific attributes like 24/7 carbon-free power.

In [32]:
HOURS_PER_YEAR = 8760  # Total hours in a year
NUCLEAR_PPA_COST_PER_MWH = 133  # 92.5 £2012/MWh in todays money
CARBON_FREE_NUCLEAR_PPA_PREMIUM = (
    NUCLEAR_PPA_COST_PER_MWH*0  # £/MWh, assumed premium for carbon-free nuclear PPA assuming 10% premium
)

cost_difference_percentages = {}
cost_microgrid = {}
cost_nuclear = {}
for load in DC_LOADS:
    n = networks[load]
    print(f"Data Centre Load: {load} MW")
    cost_of_running_microgrid_per_year = n.model.objective.value + TOTAL_ANNUITISED_INFRASTRUCTURE_COSTS
    cost_of_running_with_nuclear_ppa_per_year = (
        load
        * HOURS_PER_YEAR
        * (NUCLEAR_PPA_COST_PER_MWH + CARBON_FREE_NUCLEAR_PPA_PREMIUM)
    )  # £ per year, assuming a PPA of £60/MWh

    print(
        f"Cost of running the microgrid per year: £{cost_of_running_microgrid_per_year:,.2f}"
    )
    print(
        f"Cost of running with a nuclear PPA per year: £{cost_of_running_with_nuclear_ppa_per_year:,.2f}"
    )
    # Calculate the percentage difference in costs
    cost_difference_percentage = (
        (cost_of_running_microgrid_per_year - cost_of_running_with_nuclear_ppa_per_year)
        / cost_of_running_with_nuclear_ppa_per_year
    ) * 100
    print(f"Percentage difference in costs: {cost_difference_percentage:.2f}%\n")
    cost_difference_percentages[load] = cost_difference_percentage
    cost_microgrid[load] = cost_of_running_microgrid_per_year
    cost_nuclear[load] = cost_of_running_with_nuclear_ppa_per_year

Data Centre Load: 10 MW
Cost of running the microgrid per year: £12,897,475.36
Cost of running with a nuclear PPA per year: £11,650,800.00
Percentage difference in costs: 10.70%

Data Centre Load: 40 MW
Cost of running the microgrid per year: £32,337,769.87
Cost of running with a nuclear PPA per year: £46,603,200.00
Percentage difference in costs: -30.61%

Data Centre Load: 100 MW
Cost of running the microgrid per year: £71,218,358.88
Cost of running with a nuclear PPA per year: £116,508,000.00
Percentage difference in costs: -38.87%

Data Centre Load: 115 MW
Cost of running the microgrid per year: £80,938,506.14
Cost of running with a nuclear PPA per year: £133,984,200.00
Percentage difference in costs: -39.59%

Data Centre Load: 500 MW
Cost of running the microgrid per year: £330,422,285.64
Cost of running with a nuclear PPA per year: £582,540,000.00
Percentage difference in costs: -43.28%



In [33]:
# Create a DataFrame for cost differences
cost_difference_df = pd.DataFrame.from_dict(
    cost_difference_percentages, orient="index", columns=["Cost Difference (%)"]
).reset_index(names="Data Centre Load (MW)")

# Prepare data for extrapolation
X = cost_difference_df["Data Centre Load (MW)"].to_numpy().reshape(-1, 1)
y = cost_difference_df["Cost Difference (%)"].to_numpy()

# Plot actual data and extrapolation
fig = px.line(
    cost_difference_df,
    x="Data Centre Load (MW)",
    y="Cost Difference (%)",
    title="Cost Difference of Microgrid Scenario vs Nuclear PPA by Data Centre Load",
    labels={"Data Centre Load (MW)": "Data Centre Load (MW)", "Cost Difference (%)": "Cost Difference (%)"},
    template="plotly_white",
)

fig.update_layout(
    xaxis_title="Data Centre Load (MW)",
    yaxis_title="Cost Difference (%)",
    showlegend=True,
)
fig.show()

In [42]:
import plotly.graph_objects as go

# Select the 100 MW scenario
load = 115
microgrid_cost = cost_microgrid[load]
nuclear_cost = cost_nuclear[load]
cost_diff_pct = cost_difference_percentages[load]

# Bar chart data
scenarios = ["<b>Scenario 1</b> <i>(Baseline)</i>:<br>SMR", "<b>Scenario 2</b> <i>(Alternative)</i>:<br>Wind+BESS+Gas Backup Microgrid"]
costs = [nuclear_cost, microgrid_cost]
colors = ["#636EFA", "#00CC96"]

fig = go.Figure()

# Add bars with black outlines
for i, (scenario, cost, color) in enumerate(zip(scenarios, costs, colors)):
    fig.add_trace(
        go.Bar(
            x=[scenario],
            y=[cost],
            marker=dict(color=color, line=dict(color="black", width=3)),
            name=scenario,
            showlegend=False,
            width=0.5,
        )
    )

# Add downward arrow and annotation if microgrid is cheaper
fig.add_annotation(
    x=scenarios[1],
    y=microgrid_cost,
    text=f"{abs(cost_diff_pct):.1f}% lower",
    showarrow=True,
    arrowhead=2,
    arrowsize=1.5,
    arrowwidth=2,
    arrowcolor="black",
    ax=0,
    ay=-80,  # Negative value moves annotation above the bar
    font=dict(color="black", size=16),
    bgcolor="white",
    bordercolor="black",
    borderwidth=2,
)

fig.update_layout(
    title=f"Annual Cost Comparison: SMR vs Microgrid ({load} MW DC Load)",
    yaxis_title="Annual Cost (£)",
    xaxis_title="Scenario",
    template="plotly_white",
    bargap=0.5,
    yaxis=dict(tickformat=","),
)

fig.show()

# Adding a SMR Generator Instead 

In [35]:
def gen_model_SRM(data_centre_load_mw: float):
    # Define simulation period: a full year, hourly
    snapshots = pd.date_range(
        start="2019-01-01 00:00", end="2019-12-31 23:00", freq=MODEL_TIME_FREQUENCY
    )
    n = pypsa.Network(snapshots=snapshots)

    # --- DEFINE CARRIERS ---
    n.add("Carrier", "AC")  # For AC buses
    

    # --- DEFINE BUSES ---
    n.add("Bus", "electricity_bus", carrier="AC")

    # --- DEFINE LOAD ---
    # Load (Data Centre): The data centre is modelled as a load with a constant power demand.
    constant_demand_profile = pd.Series(data_centre_load_mw, index=n.snapshots)
    n.add(
        "Load", "data_centre_load", bus="electricity_bus", p_set=constant_demand_profile
    )

    # --- DEFINE GENERATORS ---

    # Generator (Gas Turbine Backup): The gas turbine provides backup, and its capacity is also extendable.
    n.add(
        "Generator",
        "SMR",
        bus="electricity_bus",
        carrier="AC",
        p_nom_extendable=False,  # Allow capacity to be extended to cover the data centre load
        committable=False,  # Cannot be both committable and extendable for LOPF capacity optimization. TODO: Check if this is correct
        capital_cost=2.5e9, # £ per unit, assuming a single SMR unit
        marginal_cost= 50, # £ per MWh_elec, assumed operational cost , conservative value from Izzy's estimate 
    )



    # --- CONSISTENCY CHECK & SOLVE ---

    # Perform consistency check
    try:
        n.consistency_check()
        print("Network consistency check passed before solving.")
    except Exception as e:
        print(f"Consistency check error before solving: {e}")

    return n

In [36]:
networks2 = {}
DC_LOADS = [10, 40, 100, 115, 500]  # Different data centre load scenarios in MW

for i, dc_load_value in enumerate(DC_LOADS):
    n2 = gen_model_SRM(data_centre_load_mw=dc_load_value)
    n2 = solve_model(n2, solver_name="gurobi", print_results=False)
    networks2[dc_load_value] = n2

INFO:linopy.model: Solve problem using Gurobi solver


Network consistency check passed before solving.


INFO:linopy.io: Writing time: 0.04s


Set parameter LicenseID to value 2648928


INFO:gurobipy:Set parameter LicenseID to value 2648928


...

INFO:gurobipy:


Solved in 0 iterations and 0.00 seconds (0.00 work units)


INFO:gurobipy:Solved in 0 iterations and 0.00 seconds (0.00 work units)


Infeasible or unbounded model


INFO:gurobipy:Infeasible or unbounded model
INFO:linopy.solvers:Unable to save solution file. Raised error: Unable to retrieve attribute 'X'
Status: warning
Termination condition: infeasible_or_unbounded
Solution: 0 primals, 0 duals
Objective: nan
Solver model: available
Solver message: 4



In [37]:
networks2[115].generators_t.stand_by_cost

Generator
snapshot
2019-01-01 00:00:00
2019-01-01 01:00:00
2019-01-01 02:00:00
2019-01-01 03:00:00
2019-01-01 04:00:00
...
2019-12-31 19:00:00
2019-12-31 20:00:00
2019-12-31 21:00:00


In [38]:
HOURS_PER_YEAR = 8760  # Total hours in a year
SMR_COSTS_PER_MWH =  50 # £/MWh, conservative end of the estimate given by Izzy for SMR costs
CARBON_FREE_NUCLEAR_PPA_PREMIUM = (
    SMR_COSTS_PER_MWH*0  # £/MWh, assumed premium for carbon-free nuclear PPA assuming 10% premium
)

smr_infrastructure_costs_gbp = 2.5e9 
smr_lifetime_years = 60  # Assumed lifetime of the SMR in years, from Rolls Royce : https://gda.rolls-royce-smr.com/our-technology#:~:text=Over%20its%2060%2Dyear%20lifetime,type%20of%20waste%20is%20it%3F
smr_discount_rate = 0.07 #assumed the same as wind and gas
TOTAL_ANNUITISED_INFRASTRUCTURE_COSTS_SMR = (
    (smr_infrastructure_costs_gbp * smr_discount_rate) / (1 - (1 + smr_discount_rate) ** -smr_lifetime_years))
cost_difference_percentages = {}
cost_microgrid = {}
cost_nuclear = {}
for load in DC_LOADS:
    n = networks[load]
    cost_of_running_microgrid_per_year = n.model.objective.value + TOTAL_ANNUITISED_INFRASTRUCTURE_COSTS
    print(f"Data Centre Load: {load} MW")
    print(load)
    if load < 500:
        cost_of_running_with_nuclear_ppa_per_year = (
            (load
            * HOURS_PER_YEAR
            * (SMR_COSTS_PER_MWH + CARBON_FREE_NUCLEAR_PPA_PREMIUM)) + TOTAL_ANNUITISED_INFRASTRUCTURE_COSTS_SMR
        )  # £ per year, assumping SMR of 50 £/MWh plus having built the SMR for purpose of datacentre 
    else:
        cost_of_running_with_nuclear_ppa_per_year = (
            (load
            * HOURS_PER_YEAR
            * (SMR_COSTS_PER_MWH + CARBON_FREE_NUCLEAR_PPA_PREMIUM)) + 2* TOTAL_ANNUITISED_INFRASTRUCTURE_COSTS_SMR #because for a 500 MW data centre, we need two SMRs
        )  # £ per year, assumping SMR of 50 £/MWh plus having built the SMR for purpose of datacentre 

    print(
        f"Cost of running the microgrid per year: £{cost_of_running_microgrid_per_year:,.2f}"
    )
    print(
        f"Cost of running with a purpose-built SMR per year (including capex): £{cost_of_running_with_nuclear_ppa_per_year:,.2f}"
    )
    # Calculate the percentage difference in costs
    cost_difference_percentage = (
        (cost_of_running_microgrid_per_year - cost_of_running_with_nuclear_ppa_per_year)
        / cost_of_running_with_nuclear_ppa_per_year
    ) * 100
    print(f"Percentage difference in costs: {cost_difference_percentage:.2f}%\n")
    cost_difference_percentages[load] = cost_difference_percentage
    cost_microgrid[load] = cost_of_running_microgrid_per_year
    cost_nuclear[load] = cost_of_running_with_nuclear_ppa_per_year

Data Centre Load: 10 MW
10
Cost of running the microgrid per year: £12,897,475.36
Cost of running with a purpose-built SMR per year (including capex): £182,453,063.75
Percentage difference in costs: -92.93%

Data Centre Load: 40 MW
40
Cost of running the microgrid per year: £32,337,769.87
Cost of running with a purpose-built SMR per year (including capex): £195,593,063.75
Percentage difference in costs: -83.47%

Data Centre Load: 100 MW
100
Cost of running the microgrid per year: £71,218,358.88
Cost of running with a purpose-built SMR per year (including capex): £221,873,063.75
Percentage difference in costs: -67.90%

Data Centre Load: 115 MW
115
Cost of running the microgrid per year: £80,938,506.14
Cost of running with a purpose-built SMR per year (including capex): £228,443,063.75
Percentage difference in costs: -64.57%

Data Centre Load: 500 MW
500
Cost of running the microgrid per year: £330,422,285.64
Cost of running with a purpose-built SMR per year (including capex): £575,146,1

In [39]:
# Create a DataFrame for cost differences
cost_difference_df = pd.DataFrame.from_dict(
    cost_difference_percentages, orient="index", columns=["Cost Difference (%)"]
).reset_index(names="Data Centre Load (MW)")

# Prepare data for extrapolation
X = cost_difference_df["Data Centre Load (MW)"].to_numpy().reshape(-1, 1)
y = cost_difference_df["Cost Difference (%)"].to_numpy()

# Plot actual data and extrapolation
fig = px.line(
    cost_difference_df,
    x="Data Centre Load (MW)",
    y="Cost Difference (%)",
    title="Cost Difference of Microgrid Scenario vs SMR by Data Centre Load",
    labels={"Data Centre Load (MW)": "Data Centre Load (MW)", "Cost Difference (%)": "Cost Difference (%)"},
    template="plotly_white",
)

fig.update_layout(
    xaxis_title="Data Centre Load (MW)",
    yaxis_title="Cost Difference (%)",
    showlegend=True,
)
fig.show()

In [40]:
import plotly.graph_objects as go

# Select the 100 MW scenario
load = 115
microgrid_cost = cost_microgrid[load]
nuclear_cost = cost_nuclear[load]
cost_diff_pct = cost_difference_percentages[load]

# Bar chart data
scenarios = ["<b>Scenario 1</b> <i>(Baseline)</i>:<br>Nuclear PPA", "<b>Scenario 2</b> <i>(Alternative)</i>:<br>Wind+BESS+Gas Backup Microgrid"]
costs = [nuclear_cost, microgrid_cost]
colors = ["#636EFA", "#00CC96"]

fig = go.Figure()

# Add bars with black outlines
for i, (scenario, cost, color) in enumerate(zip(scenarios, costs, colors)):
    fig.add_trace(
        go.Bar(
            x=[scenario],
            y=[cost],
            marker=dict(color=color, line=dict(color="black", width=3)),
            name=scenario,
            showlegend=False,
            width=0.5,
        )
    )

# Add downward arrow and annotation if microgrid is cheaper
fig.add_annotation(
    x=scenarios[1],
    y=microgrid_cost,
    text=f"{abs(cost_diff_pct):.1f}% lower",
    showarrow=True,
    arrowhead=2,
    arrowsize=1.5,
    arrowwidth=2,
    arrowcolor="black",
    ax=0,
    ay=-80,  # Negative value moves annotation above the bar
    font=dict(color="black", size=16),
    bgcolor="white",
    bordercolor="black",
    borderwidth=2,
)

fig.update_layout(
    title=f"Annual Cost Comparison: SMR vs Microgrid ({load} MW DC Load)",
    yaxis_title="Annual Cost (£)",
    xaxis_title="Scenario",
    template="plotly_white",
    bargap=0.5,
    yaxis=dict(tickformat=","),
)

fig.show()

# Plan for the future
1. Initialise data centre load as a constant
2. Loop through wind farms with capacities at different multiple factors of the data centre.
3. Assume constant things like capex cost and capacity factor for wind. We can do a sensitivity analysis of these later.
4. v1: Assume no battery? v2: Assume battery as a MWh of the multiple of the data centre load too.
5. Solve the model to get the least cost way of operating it with gas + battery in each setup. Calculate the % of the total data centre load that was met by gas. Crude calcualtion = MWh of gas / MWh of data centre throughout the year?


## Some options for analysis
* GIVEN a constant data centre load and GIVEN some historical wind profile. Q: How big should the battery and wind farm be priortional to the data centre load to cost-optimally run the system? How much does it cost to run that system? How would that compare to covering the data centre load via a nuclear PPA?
* Alternative route: Given different configurations of wind sizes, but allowing gas to be as much as is needed to cover the data centre load, how much does it cost to run each of those systems? Compared to nuclear PPA?